In [1]:
using Base.Threads

In [3]:
Threads.nthreads()

12

In [7]:
cd("..")

In [8]:
pwd()

"/Users/zifanyu/Documents/GitHub/BulkLMM.jl/analysis"

In [9]:
include("../test/BXDdata_for_test.jl");

In [10]:
include("../src/parallel_helpers.jl");

In [62]:
include("../test/testHelper.jl");

In [37]:
# using Pkg
# Pkg.add("ThreadTools")
using ThreadTools

In [38]:
?tmap

search: tmap tmap1 @atomicswap BitMatrix Cintmax_t floatmax @fastmath Cuintmax_t



```
tmap(f, args...)
tmap(f, v, T::DataType)
tmap(f, nthreads::Int, args...)
tmap1(f, args...)
tmap1(f, nthreads::Int, args...)
```

Threaded map. The optional argument `nthreads` limits the number of threads used in parallel. `tmap1` is the same as `tmap`, but falls back to a regular `map` if julia only has access to one thread. If the eltype `T` of the output is specified the call will be type stable.


In [40]:
function threads12_by_blocks(r0perm::Array{Float64, 2}, X00::Array{Float64, 2}, nblocks::Int64)
    # Does distributed processes of calculations of LOD scores for markers in each block

    p = size(X00, 2);

    ## (Create blocks...)
    
    block_size = ceil(Int, p/nblocks);
    blocks = createBlocks2(p, block_size);
    # blocks = createBlocks(p, nblocks);

    LODs_blocks = tmap(x -> calcLODs_block(r0perm, X00, x), 4, blocks);
    results = reduce(hcat, LODs_blocks);

    return results

end

threads12_by_blocks (generic function with 1 method)

In [41]:
function scan_perms_threads12(y::Array{Float64,2}, g::Array{Float64,2}, K::Array{Float64,2};
                                reml::Bool = false,
                                nperms::Int64 = 1024, rndseed::Int64 = 0, original::Bool = true,
                                # (options for blocks, nperms distribution methods...)
                                option::String = "by blocks", nblocks::Int64 = 1, ncopies::Int64 = 1, 
                                nprocs::Int64 = 0)

    # addprocs(nprocs);
    # @everywhere using ParallelDataTransfer
    # sendto(workers(), y = y);
    #sendto(workers(), g = g);
    #sendto(workers(), K = K);
    #sendto(workers(), reml = reml);

    #@everywhere begin
        #include("../src/parallel_helpers.jl"); # for now; needs to be revised later
        #(y0, X0, lambda0) = transform1(y, g, K); # rotation of data
        # (r0, X00) = transform2(y0, X0, lambda0; reml = reml); # reweighting and taking residuals
    
    #end

    (y0, X0, lambda0) = transform_rotation(y, g, K); # rotation of data
    (r0, X00) = transform_reweight(y0, X0, lambda0; reml = reml); # reweighting and taking residuals
    r0perm = transform_permute(r0; nperms = nperms, rndseed = rndseed, original = original);

    if option == "by blocks"
        # @everywhere r0perm = transform3(r0; nperms = nperms, rndseed = rndseed, original = original); # permutations
        results = threads12_by_blocks(r0perm, X00, nblocks);
    elseif option == "by nperms"
        results = distribute_by_nperms(r0, X00, nperms, ncopies, original);
    else
        throw(error("Option unsupported."))
    end

    return results

end

scan_perms_threads12 (generic function with 1 method)

In [58]:
@time scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 150);

  2.438899 seconds (77.84 k allocations: 13.675 GiB, 32.51% gc time)


In [60]:
tmap_LODs = scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 150);

In [45]:
using Distributed

In [49]:
# addprocs(4);

In [50]:
nprocs()

5

In [52]:
@everywhere begin
    using Random
    using LinearAlgebra
    using Optim
    using Distributed
    
    ## Include the source code of BulkLMM to be tested:
    include("../src/scan.jl");
    include("../src/lmm.jl");
    include("../src/wls.jl");
    include("../src/util.jl");
    include("../src/kinship.jl");
    include("../src/readData.jl");
    include("../src/scan_for_tests.jl")
    include("../src/scan_distributed.jl")

    include("../src/parallel_helpers.jl")
end

In [53]:
pmap_LODs = scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 12);

In [56]:
@time scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 150);

  2.374277 seconds (77.82 k allocations: 13.675 GiB, 32.41% gc time)


In [63]:
pmap_LODs

1025×7321 Matrix{Float64}:
 0.00819636   0.00819636   0.00819636   …  0.0128283    0.0128283
 1.13823      1.13823      1.13823         1.83125      1.83125
 0.182157     0.182157     0.182157        0.0104772    0.0104772
 0.0430406    0.0430406    0.0430406       0.248817     0.248817
 1.02692      1.02692      1.02692         0.41929      0.41929
 0.0273182    0.0273182    0.0273182    …  0.567742     0.567742
 0.369665     0.369665     0.369665        0.797462     0.797462
 0.0619086    0.0619086    0.0619086       0.120096     0.120096
 1.50799      1.50799      1.50799         0.124231     0.124231
 0.0214375    0.0214375    0.0214375       0.066094     0.066094
 0.457624     0.457624     0.457624     …  0.122794     0.122794
 0.0674599    0.0674599    0.0674599       0.781827     0.781827
 0.000739424  0.000739424  0.000739424     0.278836     0.278836
 ⋮                                      ⋱               ⋮
 0.305017     0.305017     0.305017        0.647157     0.647157
 0.00

In [64]:
# Check the results from tmap and pmap:
sumSqDiff(tmap_LODs, pmap_LODs)

2.5381895448325513e-26